**04. Building the inception block**

<img src = 'inception_block.png' height = '400' />

In [1]:
# DL needs
import tensorflow as tf
import keras as kr

# Data needs
import pandas as pd
from sklearn.model_selection import train_test_split

# Numerical computation needs
import numpy as np

# plotting needs
import matplotlib.pyplot as plt
import matplotlib_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('svg')

# ensuring reproducibility
random_seed=42
tf.random.set_seed(random_seed)

import warnings
warnings.filterwarnings("ignore", category=UserWarning, module="keras")



2025-04-26 13:46:18.728250: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [13]:
@kr.utils.register_keras_serializable(package='InceptionBlock')
class InceptionBlock(kr.layers.Layer):
    def __init__(self,input_channels,output_channels,expansion_factor,stride = 1,expansion_kernel_size = 1,depthwise_width = 3, depthwise_kernel_sizes = [3,3,3],depthwise_dilation_rates = [1,2,3], transpose_kernel_size=3, **kwargs):
        super().__init__(**kwargs)

        self._block_name = kwargs.get('name','InceptionBlock')
        self.stride = stride
        expanded_channels = input_channels * expansion_factor
        self.depthwise_width = depthwise_width

        # Expansion 
        self.expand_conv = tf.keras.layers.Conv2D(
            filters = expanded_channels,
            kernel_size = expansion_kernel_size,
            padding = 'same',
            use_bias = False,
            name = f'{self._block_name}_expand'
        )

        self.bn_e = tf.keras.layers.BatchNormalization(name = f'{self._block_name}_expand_BN')
        self.relu_e = tf.keras.layers.ReLU(name=f'{self._block_name}_expand_relu')

        # Depthwise convolution x 3
        self.depthwise = [
            [tf.keras.layers.DepthwiseConv2D(
                kernel_size = depthwise_kernel_sizes[0],
                strides = self.stride,
                padding = 'same',
                use_bias = False,
                name = f'{self._block_name}_depthwise_{i+1}',
                dilation_rate = depthwise_dilation_rates[0]
            ),
            tf.keras.layers.BatchNormalization(name = f'{self._block_name}_depthwise_BN_{i+1}')
            ]
            for i in range(depthwise_width)
        ]

        # Concat layer
        self.concat = tf.keras.layers.Concatenate(name =f'{self._block_name}_concat' )
        
        # Concat relu
        self.relu_concat = tf.keras.layers.ReLU(name=f'{self._block_name}_concat_relu')

        # Projection
        self.project_conv = tf.keras.layers.Conv2D(
            filters = output_channels,
            kernel_size = 1,
            padding = 'same',
            use_bias = False,
            name = f'{self._block_name}_project'
        )
        self.bn_p = tf.keras.layers.BatchNormalization(name = f'{self._block_name}_project_BN')

        # Transpose Convolution layer 
        self.transpose_conv = tf.keras.layers.Conv2DTranspose(
            filters = output_channels,
            kernel_size = transpose_kernel_size,
            strides = stride,
            padding = 'valid',
            use_bias = False,
            name = f'{self._block_name}_transpose_conv'
        )

        self.bn_tc = tf.keras.layers.BatchNormalization(name = f'{self._block_name}_transpose_conv_BN')


    def call(self,inputs,training = False):
        # expansion
        x = self.expand_conv(inputs)
        x = self.bn_e(x,training = training)
        x = self.relu_e(x)

        # depthwise convolution
        depthwise_out = []
        for depthwise_layer in self.depthwise:
            # BatchNorm(DepthwiseConv(x))
            depthwise_out.append(depthwise_layer[1](depthwise_layer[0](x),training = training))

        depthwise_out+=[x]
        
        # concatenation
        x = self.concat(depthwise_out)
        x = self.relu_concat(x)

        # projection
        x = self.project_conv(x)
        x = self.bn_p(x,training = training)

        # transpose convolution
        x = self.transpose_conv(x)
        x = self.bn_tc(x,training = training)
        
        return x

In [14]:
inputs = tf.keras.layers.Input(shape=(128,128,32))
outputs = InceptionBlock(input_channels=32,
                         output_channels=16,
                         expansion_factor=6,
                         stride = 1,
                         depthwise_width=3,
                         depthwise_kernel_sizes=[3,3,3],
                         depthwise_dilation_rates=[1,2,3],
                         transpose_kernel_size=3,
                         name = 'Inception_1')(inputs)
 
model = tf.keras.models.Model(inputs = inputs, outputs= outputs)
model.summary()


Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_1 (InputLayer)      │ (None, 128, 128, 32)   │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ Inception_1 (InceptionBlock)    │ (None, 130, 130, 16)   │        29,120 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 29,120 (113.75 KB)

 Trainable params: 27,520 (107.50 KB)

 Non-trainable params: 1,600 (6.25 KB)